In [ ]:
# TicTacToe 5x5 (4-in-a-row) with Human vs AI (Minimax + Alpha-Beta)
# Coins: 'X' (Human), 'T' (AI). Empty cells are '.'
# Win condition: any FOUR consecutive coins in a row, column, or diagonal.

from typing import List, Tuple, Optional
import math
import sys

SIZE = 5
WIN_LEN = 4
HUMAN = 'X'
AI = 'T'
EMPTY = '.'

# ======== Board Utilities ========

#defining Board 5*5
def new_board() -> List[List[str]]:
    return [[EMPTY for _ in range(SIZE)] for _ in range(SIZE)]

#Display the board
def print_board(board: List[List[str]]) -> None:
    print("\n   " + " ".join(str(c+1) for c in range(SIZE)))
    print("  " + "--" * SIZE + "-")
    for r in range(SIZE):
        print(f"{r+1}| " + " ".join(board[r]))
    print()

#Function to check if the input is within the inside board 
def InputInsideBoard(r: int, c: int) -> bool:
    if(0<=r<SIZE and 0<=c< SIZE):
        return True
    else :
        return False
    


def get_valid_moves(board: List[List[str]]) -> List[Tuple[int, int]]:
    moves = [(r, c) for r in range(SIZE) for c in range(SIZE) if board[r][c] == EMPTY]
    # Move ordering: prefer center and cells near existing stones
    center = (SIZE-1)/2.0
    def heuristic(m: Tuple[int,int]) -> float:
        r, c = m
        # closeness to center
        center_dist = abs(r - center) + abs(c - center)
        # adjacency bonus: count neighbors that are non-empty
        adj = 0
        for dr in (-1, 0, 1):
            for dc in (-1, 0, 1):
                if dr == 0 and dc == 0:
                    continue
                nr, nc = r+dr, c+dc
                if InputInsideBoard(nr, nc) and board[nr][nc] != EMPTY:
                    adj += 1
        # Lower score is better; more adjacency (negative) and closer to center (negative)
        return center_dist - 0.25 * adj
    moves.sort(key=heuristic)
    return moves


# ======== Win / Terminal Checks ========

DIRECTIONS = [
    (0, 1),   # right
    (1, 0),   # down
    (1, 1),   # down-right
    (1, -1),  # down-left
]


def has_k_in_a_row(board: List[List[str]], r: int, c: int, dr: int, dc: int, player: str, k: int) -> bool:
    for i in range(k):
        rr, cc = r + dr * i, c + dc * i
        if not InputInsideBoard(rr, cc) or board[rr][cc] != player:
            return False
    return True


def winner(board: List[List[str]]) -> Optional[str]:
    for r in range(SIZE):
        for c in range(SIZE):
            if board[r][c] == EMPTY:
                continue
            p = board[r][c]
            for dr, dc in DIRECTIONS:
                if has_k_in_a_row(board, r, c, dr, dc, p, WIN_LEN):
                    return p
    return None


def is_full(board: List[List[str]]) -> bool:
    return all(board[r][c] != EMPTY for r in range(SIZE) for c in range(SIZE))


# ======== Static Evaluation Function ========
# Count and score all sliding windows of length WIN_LEN along rows, columns, and diagonals.
# Scoring philosophy (no ML):
#   - 4 in a row  -> Huge +/- value (decided as terminal elsewhere, but kept for robustness)
#   - 3 + 1 empty -> strong threat
#   - 2 + 2 empty -> moderate potential
#   - 1 + 3 empty -> small potential
#   - mixed (both players present) -> neutral (0)

SCORES = {
    (4, 0): 100000,  # four of AI with 0 opponent -> win
    (3, 0): 100,     # three of AI + one empty
    (2, 0): 10,
    (1, 0): 1,
}

# Apply symmetrical penalty for opponent patterns.


def evaluate_window(window: List[str], ai: str, opp: str) -> int:
    ai_count = window.count(ai)
    opp_count = window.count(opp)
    empty_count = window.count(EMPTY)
    # If both players appear, window cannot contribute to a single side
    if ai_count > 0 and opp_count > 0:
        return 0
    if ai_count > 0 and opp_count == 0:
        return SCORES.get((ai_count, 0), 0)
    if opp_count > 0 and ai_count == 0:
        return -SCORES.get((opp_count, 0), 0)
    return 0  # all empty


def evaluate(board: List[List[str]], player: str = AI) -> int:
    ai, opp = player, (HUMAN if player == AI else AI)
    score = 0

    # Rows
    for r in range(SIZE):
        for c in range(SIZE - WIN_LEN + 1):
            window = [board[r][c+i] for i in range(WIN_LEN)]
            score += evaluate_window(window, ai, opp)

    # Columns
    for c in range(SIZE):
        for r in range(SIZE - WIN_LEN + 1):
            window = [board[r+i][c] for i in range(WIN_LEN)]
            score += evaluate_window(window, ai, opp)

    # Diagonals (down-right)
    for r in range(SIZE - WIN_LEN + 1):
        for c in range(SIZE - WIN_LEN + 1):
            window = [board[r+i][c+i] for i in range(WIN_LEN)]
            score += evaluate_window(window, ai, opp)

    # Diagonals (down-left)
    for r in range(SIZE - WIN_LEN + 1):
        for c in range(WIN_LEN - 1, SIZE):
            window = [board[r+i][c-i] for i in range(WIN_LEN)]
            score += evaluate_window(window, ai, opp)

    return score


# ======== Minimax with Alpha-Beta ========


def minimax(board, depth, alpha, beta, maximizingPlayer):
    score = evaluate(board)
    if abs(score) == 1000 or depth == 0 or is_full(board):
        return score

    if maximizingPlayer:  # AI's turn
        maxEval = -math.inf
        for (r, c) in get_moves(board):
            board[r][c] = AI
            eval = minimax(board, depth - 1, alpha, beta, False)
            board[r][c] = EMPTY
            maxEval = max(maxEval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval
    else:  # Human's turn
        minEval = math.inf
        for (r, c) in get_moves(board):
            board[r][c] = HUMAN
            eval = minimax(board, depth - 1, alpha, beta, True)
            board[r][c] = EMPTY
            minEval = min(minEval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval


# ======== Game Loop ========

def ask_int(prompt: str, lo: int, hi: int) -> int:
    while True:
        try:
            val = int(input(prompt))
            if lo <= val <= hi:
                return val
            print(f"Enter a number between {lo} and {hi}.")
        except ValueError:
            print("Please enter an integer.")


def ask_int(prompt: str, min_val: int, max_val: int) -> int:
    """Ask user for an integer input restricted to range."""
    while True:
        try:
            value = int(input(prompt))  # restrict to board size
            if min_val <= value <= max_val:
                return value
            else:
                print(f"Please enter a number between {min_val} and {max_val}.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

def human_move(board: List[List[str]]) -> None:
    """Ask human player to make a valid move on the board."""
    while True:
        r = ask_int(f"Row (1-{SIZE}): ", 1, SIZE) - 1
        c = ask_int(f"Col (1-{SIZE}): ", 1, SIZE) - 1
        if board[r][c] == EMPTY:
            board[r][c] = HUMAN
            return

# Get available moves
def get_moves(board):
    return [(r, c) for r in range(SIZE) for c in range(SIZE) if board[r][c] == EMPTY]

def ai_move(board: List[List[str]]) -> None:
    bestVal = -math.inf
    move = None
    for (r, c) in get_moves(board):
        board[r][c] = AI
        moveVal = minimax(board, 3, -math.inf, math.inf, False)  # depth = 3
        board[r][c] = EMPTY
        if moveVal > bestVal:
            bestVal = moveVal
            move = (r, c)
    if move:
        r, c = move
        board[r][c] = AI
        print(f"AI plays at (row {r+1}, col {c+1})")


def play():
    board = new_board()
    print_board(board)

    # Choose who starts
    first = input("Do you want to move first? (y/n) ").strip().lower().startswith('y')

    while True:
        if first:
            # Human turn
            human_move(board)
            print_board(board)
            w = winner(board)
            if w or is_full(board):
                break
            # AI turn
            ai_move(board)
            print_board(board)
        else:
            # AI first
            ai_move(board)
            print_board(board)
            w = winner(board)
            if w or is_full(board):
                break
            human_move(board)
            print_board(board)

        w = winner(board)
        if w or is_full(board):
            break

    w = winner(board)
    if w == HUMAN:
        print("You win! 🎉")
    elif w == AI:
        print("AI wins! 🤖")
    else:
        print("It's a draw.")


if __name__ == "__main__":
    try:
        play()
    except KeyboardInterrupt:
        print("\nGame interrupted.")
        sys.exit(0)
